<span style='font-size:150%'>딥러닝 수행 (10점) - TensorFlow, Keras를 활용하여 딥러닝 수행</span>   
  
다양한 방식으로 딥러닝을 수행한 코딩을 모두 제출할 것    
   
파일명 : 4_Deep learning_1.ipynb   
   
(여러개의 파일을 사용한 경우 Deep learning_2.ipynb, Deep learning_3.ipynb...)   

# 참고
딥러닝 모델이 총 12개가 있습니다.   
맨 첫 번째 모델에 대한 ipynb 파일을 복사하여 그대로 이용하였고, window_size, batch_size의 숫자만 다르게 해서 딥러닝을 진행했습니다.  
혹시나 전부 보시느라 시간을 너무나 많이 쓰실까봐 말씀드립니다!! 첫 번째, 두 번째 정도?? 만 보셔도 될 것 같습니다.


그리고 ipynb 파일 이름에 _(숫자, 숫자) 이런식으로 쓴 건, (window_size, batch_size) 입니다!!

In [1]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import matplotlib.font_manager

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

seed=0
np.random.seed(3)
tf.random.set_seed(3)

In [2]:
import os
os.getcwd()

'D:\\ai\\Final'

# 데이터
Data_set_1 : 종가,시가, 고가, 저가, 거래량, 변동 데이터가 담김. 아직 클래스 데이터 문자 형태  
Data_set_2 : 기존 속성 데이터들을 속성들 간의 차로 변경. 스케일링됨. 클래스 데이터 숫자 형태  

In [3]:
# 데이터 불러오기

# 일반 데이터
data_set=pd.read_csv("./Data set_2.csv")

In [4]:
data_set

,close-start,high-low,high-close,high-start,close-low,start-low,volume,Diff
0,0.541985,0.033708,0.067568,0.084337,0.051282,0.024390,0.111197,0
1,0.465649,0.168539,0.121622,0.012048,0.153846,0.243902,0.123828,1
2,0.465649,0.112360,0.202703,0.084337,0.012821,0.109756,0.134799,1
3,0.633588,0.101124,0.000000,0.168675,0.192308,0.012195,0.129628,0
4,0.412214,0.112360,0.202703,0.000000,0.012821,0.195122,0.059433,1
...,...,...,...,...,...,...,...,...
594,0.603053,0.292135,0.162162,0.265060,0.256410,0.121951,0.437442,0
595,0.450382,0.224719,0.324324,0.168675,0.025641,0.146341,0.374049,1
596,0.801527,0.359551,0.000000,0.433735,0.487179,0.024390,0.357297,0
597,0.557252,0.224719,0.243243,0.265060,0.102564,0.048780,0.644365,1


# 데이터셋 설정

주가를 예측하는 것은 시계열 데이터를 필요로 하기 때문에 기존 데이터를 시계열 데이터에 맞게 변형을 해줘야 한다.  
며칠 동안의 데이터를 가지고 예측을 할지를 window_size에 입력받고, 해당 숫자를 기반으로 시계열 데이터셋를 생성할 것이다.  
window_size에 따라 딥러닝 결과가 달라질 수 있기 때문에 시계열 데이터를 '2_...'나 '3_...'에서 만들지 않고 '4_...'에서 바로바로 만들어서 사용했다.  

예를 들자면 다음과 같다.

시계열 데이터 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  

window_size = 4 이면

시계열 데이터셋 = [ [1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6], [4, 5, 6, 7], [5, 6, 7, 8], [6, 7, 8, 9], [7, 8, 9, 10] ]

또한 여러번 실행을 해보기 위해 함수를 사용했다.

In [5]:
# window의 사이즈와 train dataset의 사이즈 입력받는 함수

def set_window_train_size(data):
    
    # 학습 데이터셋 70% 테스트 데이터셋 30%로 나눈다. (고정)
    train_size=int(len(data)*0.7) 
    
    # 시계열 데이터셋을 만들 때, 몇 개의 데이터를 가지고 만들지 설정한다.
    # 예 : window_size=30 => 30일간의 데이터를 가진 시계열 데이터셋
    window_size=int(input("window size : "))

    return window_size, train_size

In [6]:
# feature 데이터와 class 데이터를 구분

def split_feature_class(data):
    
    #feature_cols = ['close-start', 'high-low', 'high-close', 'high-start','close-low','start-low','volume']
    #close_cols = ['close']
    
    feature_cols = ['close-start', 'high-low', 'high-close', 'high-start','close-low','start-low','volume']
    close_cols = ['Diff']
    
    data_feature = data[feature_cols]
    data_close = data[close_cols]
    
    return data_feature, data_close

In [7]:
# window_size에 따라 데이터를 묶어주고 새로운 데이터셋 리턴
def make_window_data(feature_data, class_data, window_size):
    
    feature_list=[]
    class_list=[]
    
    for i in range(len(feature_data)-window_size):
        feature_list.append(np.array(feature_data.iloc[i:i+window_size]))
        class_list.append(np.array(class_data.iloc[i+window_size]))
    
    return np.array(feature_list), np.array(class_list)

In [8]:
def make_dataset(data):
    
    # 시계열 데이터 크기(window_size)와 학습 데이터(train_size)의 크기
    window_size, train_size=set_window_train_size(data)
    
    # 학습 데이터셋과 테스트 데이터셋 분리
    train = data[:train_size] 
    test = data[train_size:] 
    
    train_feature, train_class = split_feature_class(train)
    test_feature, test_class = split_feature_class(test)
    
    train_feature, train_class = make_window_data(train_feature, train_class, window_size)
    test_feature, test_class = make_window_data(test_feature, test_class, window_size)
    
    return train_feature, train_class, test_feature, test_class, window_size

In [9]:
# 학습 데이터셋을 train과 valid로 분리

def train_valid(train_feature,train_class):
    split_size= float(input("학습 데이터 중 test size(validation data의 size)의 비율 (예 : 0.2) : "))
    X_train, X_valid, Y_train, Y_valid = train_test_split(train_feature, train_class, test_size = split_size, random_state=seed)
    return X_train, X_valid, Y_train, Y_valid

# 기본 데이터에 대한 딥러닝

In [10]:
train_feature, train_class, test_feature, test_class, window_size = make_dataset(data_set)

X_train, X_valid, Y_train, Y_valid = train_valid(train_feature, train_class)

window size : 5
학습 데이터 중 test size(validation data의 size)의 비율 (예 : 0.2) : 0.2


# 모델 설정

<span style='font-size:150%'>[LSTM]</span>  
  
시계열 데이터이기 때문에 LSTM을 사용했다.  

2진 class(0 or 1)      
활성화 함수 : 'sigmoid'   
손실 함수 : 'binary_crossentropy'  

In [43]:
model = Sequential()
model.add(LSTM(20, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), #시계열 데이터셋의 구조
               activation='relu',return_sequences=False))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

#optimizer='rmsprop'
#optimizer='adam'

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20)                2240      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 2,261
Trainable params: 2,261
Non-trainable params: 0
_________________________________________________________________


# 모델 업데이트 과정 저장을 위한 환경 설정

In [56]:
# 모델 저장 폴더 생성
model_dir='./model_save'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
# 파일 저장 경로
# 수업 시간에 다뤘던 파일 경로 및 이름에 'window_size'를 추가하여 응용해보았습니다.
# 경로+딥러닝 모델 순서+window_size+epoch+val_loss+val_accuracy
model_path='./model_save/1_'+str(window_size)+'_{epoch:d}_{val_loss:.4f}_{val_accuracy:.4f}.h5'

In [45]:
model_path

'./model_save/1_5_{epoch:d}_{val_loss:.4f}_{val_accuracy:.4f}.h5'

In [46]:
# 학습 자동 중단 설정
# 20번 연속 성능에 변화 없으면 학습 멈춤
early_stop = EarlyStopping(monitor='val_loss', patience=20)

In [47]:
# 모델 업데이트 및 저장
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

# 딥러닝

In [48]:
history = model.fit(X_train, Y_train,                     
                    validation_data=(X_valid, Y_valid), 
                    epochs=200, 
                    batch_size=10, 
                    callbacks=[early_stop, checkpoint])

Train on 331 samples, validate on 83 samples
Epoch 1/200
280/331 [========================>.....] - ETA: 0s - loss: 0.6935 - accuracy: 0.5107
Epoch 00001: val_loss improved from inf to 0.69633, saving model to ./model_save/1_5_1_0.6963_0.4578.h5
331/331 [==============================] - 4s 13ms/sample - loss: 0.6930 - accuracy: 0.5166 - val_loss: 0.6963 - val_accuracy: 0.4578
Epoch 2/200
310/331 [===========================>..] - ETA: 0s - loss: 0.6934 - accuracy: 0.5065
Epoch 00002: val_loss did not improve from 0.69633
331/331 [==============================] - 0s 1ms/sample - loss: 0.6926 - accuracy: 0.5196 - val_loss: 0.6972 - val_accuracy: 0.4578
Epoch 3/200
290/331 [=========================>....] - ETA: 0s - loss: 0.6917 - accuracy: 0.5310
Epoch 00003: val_loss improved from 0.69633 to 0.69572, saving model to ./model_save/1_5_3_0.6957_0.4578.h5
331/331 [==============================] - 0s 1000us/sample - loss: 0.6926 - accuracy: 0.5166 - val_loss: 0.6957 - val_accuracy: 0.457

300/331 [==========================>...] - ETA: 0s - loss: 0.6894 - accuracy: 0.5200
Epoch 00028: val_loss did not improve from 0.69115
331/331 [==============================] - 0s 717us/sample - loss: 0.6891 - accuracy: 0.5257 - val_loss: 0.6935 - val_accuracy: 0.4819
Epoch 29/200
300/331 [==========================>...] - ETA: 0s - loss: 0.6880 - accuracy: 0.5500
Epoch 00029: val_loss did not improve from 0.69115
331/331 [==============================] - 0s 868us/sample - loss: 0.6887 - accuracy: 0.5408 - val_loss: 0.6937 - val_accuracy: 0.4819
Epoch 30/200
290/331 [=========================>....] - ETA: 0s - loss: 0.6885 - accuracy: 0.5483
Epoch 00030: val_loss did not improve from 0.69115
331/331 [==============================] - 0s 874us/sample - loss: 0.6892 - accuracy: 0.5408 - val_loss: 0.6944 - val_accuracy: 0.4819
Epoch 31/200
290/331 [=========================>....] - ETA: 0s - loss: 0.6865 - accuracy: 0.5345
Epoch 00031: val_loss did not improve from 0.69115
331/331 [===

331/331 [==============================] - 0s 895us/sample - loss: 0.6699 - accuracy: 0.5801 - val_loss: 0.7050 - val_accuracy: 0.5301
Epoch 84/200
290/331 [=========================>....] - ETA: 0s - loss: 0.6663 - accuracy: 0.6034
Epoch 00084: val_loss did not improve from 0.67805
331/331 [==============================] - 0s 880us/sample - loss: 0.6670 - accuracy: 0.5952 - val_loss: 0.6825 - val_accuracy: 0.5301
Epoch 85/200
290/331 [=========================>....] - ETA: 0s - loss: 0.6688 - accuracy: 0.5931
Epoch 00085: val_loss did not improve from 0.67805
331/331 [==============================] - 0s 850us/sample - loss: 0.6698 - accuracy: 0.5921 - val_loss: 0.6847 - val_accuracy: 0.4819
Epoch 86/200
230/331 [===================>..........] - ETA: 0s - loss: 0.6712 - accuracy: 0.5870
Epoch 00086: val_loss did not improve from 0.67805
331/331 [==============================] - 0s 588us/sample - loss: 0.6681 - accuracy: 0.5921 - val_loss: 0.7000 - val_accuracy: 0.5542


In [49]:
print("\nAccuracy : %.4f"%(model.evaluate(train_feature,train_class)[1]))

414/414 [==============================] - 0s 107us/sample - loss: 0.6721 - accuracy: 0.5894

Accuracy : 0.5894


In [50]:
print("\nAccuracy : %.4f"%(model.evaluate(test_feature,test_class)[1]))

175/175 [==============================] - 0s 202us/sample - loss: 0.7184 - accuracy: 0.5486

Accuracy : 0.5486


나름 만족. 후보 1.

파일 이름  
1_5_66_0.6780_0.5422.h5

# 딥러닝 결과 저장(history 저장)

In [51]:
result=pd.DataFrame(history.history)
result

,loss,accuracy,val_loss,val_accuracy
0,0.693012,0.516616,0.696327,0.457831
1,0.692625,0.519637,0.697195,0.457831
2,0.692603,0.516616,0.695723,0.457831
3,0.692209,0.519637,0.694532,0.469880
4,0.692148,0.537764,0.694106,0.481928
...,...,...,...,...
81,0.668287,0.592145,0.689639,0.566265
82,0.669924,0.580060,0.705023,0.530120
83,0.667029,0.595166,0.682539,0.530120
84,0.669817,0.592145,0.684660,0.481928


In [52]:
file_name=input("딥러닝 결과를 csv 타입으로 저장할 경로와 파일명은? (예 : D:\\ai\\Final\\history\\result_1.csv) : ")

result.to_csv(file_name,encoding='utf-8-sig',index=False)

딥러닝 결과를 csv 타입으로 저장할 경로와 파일명은? (예 : D:\ai\Final\history\result_1.csv) : D:\ai\Final\history\result_1.csv
